In [7]:
from general_tools.in_out.basics import files_in_subdirs, copy_folder_structure, create_dir
from geo_tool import Point_Cloud
import os.path as osp
from multiprocessing import Pool
from tf_lab.autopredictors.scripts.virt_scan_data import pc_sampler

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [23]:
# Parameters
n_threads = 50
n_pc_samples = 128

syn_id = '03001627' # if is None then all classes will be used.
top_in_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Distance_Fields/shapenet_dim32_sdf_pc'
top_out_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Point_Clouds/Partial_PCs/Angela_synthetic'
top_out_dir = osp.join(top_out_dir, str(n_pc_samples) + osp.sep)

if syn_id is not None:
    top_in_dir = osp.join(top_in_dir, syn_id)
    top_out_dir = osp.join(top_out_dir, syn_id)
    create_dir(top_out_dir)
else:
    copy_folder_structure(top_in_dir, top_out_dir)

In [24]:
ending = '.ply'
search_pattern = ending + '$'
all_in_files = [f for f in files_in_subdirs(top_in_dir, search_pattern)]
print len(all_in_files)

48000


In [25]:
drop = len(ending)
pool = Pool(n_threads)
print 'Starting a pool with %d workers.' % (n_threads, )

for in_f in all_in_files:
    out_f = in_f.replace(top_in_dir, top_out_dir)
    out_f = out_f[:-drop]
    pool.apply_async(pc_sampler, (in_f, n_pc_samples, out_f))
    
pool.close()
pool.join()

Starting a pool with 50 workers.


In [26]:
# run this to verify the produced number of .ply files matches those read at the in_dir.
all_in_files = [f for f in files_in_subdirs(top_out_dir, '.ply')]
print len(all_in_files)

48000
